<h1><b><u>Fruits Classifier</u></b></h1>

In [ ]:
#Setting up resources and libraries required

import matplotlib.pyplot as plt
%matplotlib inline
import keras
import numpy
import itertools
from keras.models import Sequential
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolution import *
from sklearn.metrics import confusion_matrix


Using TensorFlow backend.


In [ ]:
#Local and relative paths to the files are used..
train_path = 'C:/Users/DesignerBe/AnacondaProjects/Fruits/DataImages/Train'
valid_path = 'C:/Users/DesignerBe/AnacondaProjects/Fruits/DataImages/Validate'
test_path = 'C:/Users/DesignerBe/AnacondaProjects/Fruits/DataImages/Test'

In [ ]:
train_bconvert = ImageDataGenerator().flow_from_directory(train_path,target_size=(224,224),classes=['Mangoe','Orange'],batch_size=10)
valid_bconvert = ImageDataGenerator().flow_from_directory(valid_path,target_size=(224,224),classes=['Mangoe','Orange'],batch_size=4)
test_bconvert = ImageDataGenerator().flow_from_directory(test_path,target_size=(224,224),classes=['Mangoe','Orange'],batch_size=10)

In [ ]:
#Plotting labelled images
def plots(ims,figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if(ims.shape[-1] !=3):
            ims = ims.transpose((0,2,3,1))
            f = plt.figure(figsize=figsize)
            cols = len(ims)//rows if len(ims % 2 ==0 else len(ims)//rows) +1
            for i in range(len(ims)):
                sp = f.add_subplot(rows,cols,i+1)
                sp.axis('Off')
                if titles is not None:
                    sp.set_title(titles[i], fontsize=16)
                plt.imshow(ims[i], interpolation= None if interp else 'none')

In [ ]:
imgs, labels = next(train_bconvert)

In [ ]:
plots(imgs, titles=labels)

<h2>The Convolution Neural Network:</h2>

In [ ]:
model = Sequential([
        Conv2D_1(32, (3, 3), activation='relu', input_shape=(224,224,3)),
        Flatten(),
        Dense(2, activation='softmax'),
    ])

In [ ]:
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit_generator(train_bconvert, steps_per_epoch=4, 
                    validation_data=valid_bconvert, validation_steps=4, epochs=5, verbose=2)

<h3>Predicting Now</h3>

In [ ]:
test_imgs, test_labels = next(test_bconvert)
plots(test_imgs, titles=test_labels)

In [ ]:
test_labels = test_labels[:,0]
test_labels

In [ ]:
predictions = model.predict_generator(test_bconvert, steps=1, verbose=0)

In [ ]:
predictions

In [ ]:
cm = confusion_matrix(test_labels, predictions[:,0])

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cm_plot_labels = ['Mangoe','Orange']
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')

<h2>Fine-tuning VGG16 image classifier model </h2>

In [ ]:
vgg16_model = keras.applications.vgg16.VGG16()

In [ ]:
vgg16_model.summary()

In [ ]:
model = Sequential()
for layer in vgg16_model.layers:
    model.add(layer)

In [ ]:
model.summary()

In [ ]:
model.layers.pop()

In [ ]:
for layer in model.layers:
    layer.trainable = False

In [ ]:
model.add(Dense(2, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_bconvert, steps_per_epoch=4, 
                    validation_data=valid_bconvert, validation_steps=4, epochs=5, verbose=2)

In [ ]:
test_imgs, test_labels = next(test_bconvert)
plots(test_imgs, titles=test_labels)

In [ ]:
test_labels = test_labels[:,0]
test_labels

In [ ]:
predictions = model.predict_generator(test_batches, steps=1, verbose=0)

In [ ]:
cm = confusion_matrix(test_labels, np.round(predictions[:,0]))

In [ ]:
cm_plot_labels = ['Mangoe','Orange']
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')